In [104]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [49]:
PROJECT_ID = "*******"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [50]:
import IPython.display
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

In [3]:
from google.cloud import storage
import deeplake
import random
import os
import time
from PIL import Image as PILImage
from IPython.display import Image

/opt/conda/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.0.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [4]:
MODEL_ID = "gemini-1.5-flash-002"  # @param {type:"string"}

model = GenerativeModel(MODEL_ID)

In [5]:
def create_bucket(bucket_name):
    """Creates a new bucket."""
    # bucket_name = "your-new-bucket-name"

    storage_client = storage.Client()

    bucket = storage_client.create_bucket(bucket_name)

    print(f"Bucket {bucket.name} created")

def upload_image_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """
    Uploads an image file to a Google Cloud Storage bucket.
    
    Args:
        bucket_name (str): The name of the bucket to upload to.
        source_file_name (str): The local path to the image file.
        destination_blob_name (str): The name for the image in the bucket.
    """
    # Initialize a client
    storage_client = storage.Client()
    
    # Get the bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Create a blob in the bucket
    blob = bucket.blob(destination_blob_name)
    
    # Upload the file to the blob
    blob.upload_from_filename(source_file_name)
    
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

def get_public_url(bucket_name, destination_blob_name):
    """
    Constructs a public URL for an image in a Google Cloud Storage bucket.
    
    Args:
        bucket_name (str): The name of the bucket.
        destination_blob_name (str): The name of the image in the bucket.
    
    Returns:
        str: The public URL for the image.
    """
    return f"https://storage.googleapis.com/{bucket_name}/{destination_blob_name}"


In [6]:
#create_bucket('capstone-hripsime-voskanyan')

In [7]:
#os.makedirs("wikiart_images")

In [6]:
output_dir = "wikiart_images"

In [7]:
bucket_name = "capstone-hripsime-voskanyan"
prompt = f"""
  Describe the painting with what is painted there
"""

In [8]:
ds = deeplake.load("hub://activeloop/wiki-art")

\

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/wiki-art



|

hub://activeloop/wiki-art loaded successfully.



In [9]:
prompts = []

In [44]:
random.seed(711)

In [54]:
random_indices = random.sample(range(len(ds)), 1)

In [55]:
import os
import time
from PIL import Image as PILImage

# Assuming other necessary imports and functions like upload_image_to_gcs, get_public_url, etc., are defined

# Load the WikiArt dataset from DeepLake
for idx in random_indices:
    image_path = os.path.join(output_dir, f"{idx}.png")  # Construct full image path
    
    # Check if the image already exists in the output directory
    if os.path.exists(image_path):
        print(f"Image {image_path} already exists. Skipping...")
        continue  # Skip processing this image

    # Access image data
    image = ds[idx]["images"].numpy()
    image = PILImage.fromarray(image)  # Convert to PIL Image for saving

    # Save the image
    image.save(image_path)
    print(f"Image {image_path} saved.")

    # Upload the image to GCS and get the public URL
    upload_image_to_gcs(bucket_name, image_path, os.path.basename(image_path))
    public_url = get_public_url(bucket_name, os.path.basename(image_path))
    image_file_uri = f"gs://{public_url[31:]}"
    image_file = Part.from_uri(image_file_uri, mime_type="image/png")
    contents = [prompt, image_file]

    # Generate content using the model
    response = model.generate_content(contents)
    prompts.append(response.text)

    # Log the response to a file
    with open('response_log3.txt', 'a') as f:
        f.write(response.text + '\n')  # Write response text to file
        f.write('-' * 50 + '\n')

    # Pause to avoid overloading resources
    time.sleep(30)


Image wikiart_images/72448.png saved.
File wikiart_images/72448.png uploaded to 72448.png.


In [48]:
from google.cloud import storage
import os

def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """
    Uploads a file to Google Cloud Storage.

    Args:
        bucket_name (str): The name of the GCS bucket.
        source_file_path (str): The local path to the file to upload.
        destination_blob_name (str): The destination name of the file in the bucket.
    """
    # Initialize a GCS client
    client = storage.Client()
    
    # Get the bucket
    bucket = client.bucket(bucket_name)
    
    # Create a blob and upload the file
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)

    print(f"File {source_file_path} uploaded to gs://{bucket_name}/{destination_blob_name}")

# Example usage
bucket_name = "capstone-hripsime-voskanyan"  # Replace with your bucket name
source_file_path = "poisoned_images.zip"  # Replace with your .zip file path
destination_blob_name = "poisoned_images.zip"  # Replace with desired file name in the bucket

upload_to_gcs(bucket_name, source_file_path, destination_blob_name)


File poisoned_images.zip uploaded to gs://capstone-hripsime-voskanyan/poisoned_images.zip
